In [1]:
####### module import 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
# import seaborn as sns
# Suppress Deprecation and Incorrect Usage Warnings 
import warnings
import scipy.io as sio 
import os
import scipy.signal as signal
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer , MinMaxScaler
from sklearn.decomposition import PCA
warnings.filterwarnings('ignore')
import copy
import scipy.signal as signal
import scipy.stats as stats
import scipy.io as sio
import tqdm
from torch.utils.data import Dataset

In [45]:
data = np.load('./data/1.0.npy')

In [46]:
data.shape

(5, 2400)

In [48]:
data[0,:]

array([14.66085519, 14.65925504, 14.64617424, ..., 14.18128998,
       14.18512493, 14.20130588])

In [49]:
_,_, spec_data = signal.spectrogram(data[0,:],fs=1200,nperseg=256,noverlap=128,nfft=1024)
spec_data

array([[1.08278797e-05, 5.53920174e-06, 2.29579565e-06, ...,
        5.24512813e-08, 4.68298894e-07, 3.39861850e-07],
       [3.57938324e-05, 4.82053202e-05, 1.87760579e-07, ...,
        3.52065664e-04, 1.30183380e-04, 3.60864259e-04],
       [7.26301702e-05, 2.06384163e-04, 2.00519167e-05, ...,
        1.08884221e-03, 4.47711797e-04, 1.11777081e-03],
       ...,
       [1.79993368e-08, 1.76072544e-07, 1.20783217e-07, ...,
        7.49058838e-08, 2.71896835e-08, 1.36127625e-08],
       [5.34422544e-08, 2.48106291e-07, 8.28063885e-08, ...,
        4.46004132e-08, 4.71601249e-08, 6.88080692e-09],
       [3.57734101e-08, 1.38451283e-07, 3.18002008e-08, ...,
        1.33314176e-08, 2.77322118e-08, 2.99547607e-09]])

In [94]:
data_set = np.load(f'./data/{0}.0.npy')
_,_, spec_data1 = signal.spectrogram(data_set[0,:],fs=1200,nperseg=64,noverlap=32,nfft=256)
_,_, spec_data2 = signal.spectrogram(data_set[1,:],fs=1200,nperseg=64,noverlap=32,nfft=256)
_,_, spec_data3 = signal.spectrogram(data_set[2,:],fs=1200,nperseg=64,noverlap=32,nfft=256)
_,_, spec_data4 = signal.spectrogram(data_set[3,:],fs=5000,nperseg=64,noverlap=32,nfft=256)

spec_data1 = spec_data1[:240,:]
spec_data2 = spec_data2[:240,:]
spec_data3 = spec_data3[:240,:]
spec_data4 = spec_data4[:240,:]

data = np.stack([spec_data1, spec_data2, spec_data3, spec_data4], axis=0)
data = np.squeeze(data).shape
# return data,target
target = data_set[4][0]
target

3.0

In [103]:
import copy
import scipy.signal as signal
import scipy.stats as stats
import scipy.io as sio
import tqdm
from torch.utils.data import Dataset

class Dataset(Dataset):
    def __init__(self, path):
        self.path = path
        if self.path[-1] != '/':
            self.path += '/'        
        self.NFFF = 200

    def __len__(self):
        return 179

    def __getitem__(self, idx):
        data_set = np.load(f'{self.path}{idx}.0.npy')
        _,_, spec_data1 = signal.spectrogram(data_set[0,:],fs=1200,nperseg=64,noverlap=32,nfft=256)
        _,_, spec_data2 = signal.spectrogram(data_set[1,:],fs=1200,nperseg=64,noverlap=32,nfft=256)
        _,_, spec_data3 = signal.spectrogram(data_set[2,:],fs=1200,nperseg=64,noverlap=32,nfft=256)
        _,_, spec_data4 = signal.spectrogram(data_set[3,:],fs=5000,nperseg=64,noverlap=32,nfft=256)

        spec_data1 = spec_data1[:240,:]
        spec_data2 = spec_data2[:240,:]
        spec_data3 = spec_data3[:240,:]
        spec_data4 = spec_data4[:240,:]
        data = np.stack([spec_data1, spec_data2, spec_data3, spec_data4], axis=0)
        data = np.squeeze(data).shape
        target = data_set[4][0]
        return data,target
        

In [104]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class NN(nn.Module):
    def __init__(self,NFILT=3,NOUT=4):
        super(NN,self).__init__()
        self.conv0 = nn.Conv2d(4,NFILT, kernel_size=(2,2))
        # self.conv0 = nn.Conv2d(1,NFILT,kernel_size=(200,3),padding=(0,1),bias=False)
        # self.bn0 = nn.BatchNorm2d(NFILT)
        self.gru = nn.GRU(input_size=NFILT,hidden_size=4,num_layers=1,batch_first=True,bidirectional=False)
        self.fc1 = nn.Linear(3,NOUT)



    def forward(self, x):
        x = F.relu(self.conv0(x))
        # x = x.squeeze().permute(0,2,1)
        x,_ = self.gru(x)
        x = F.dropout(x,p=0.5,training=self.training)
        x = self.fc1(x)
        return x

EPOCH:00	ITER:00000	LOSS:36.72586441040039


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x1 and 3x32)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class NN(nn.Module):
    def __init__(self,NFILT=32,NOUT=4):
        super(NN,self).__init__()
        self.h1 = nn.Linear(3, NFILT)
        # self.h2 = nn.Linear(NFILT, 10)
        self.out = nn.Linear(NFILT,NOUT)

    def forward(self, x):
        x = F.relu(self.h1(x))
        # x = F.relu(self.h2(x))
        # x = nn.Linear(32, )
        x = self.out(x)
        return x


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

epochs = 5
lr = 0.01
momentum = 0.5
log_interval = 200
seed = 1
torch.manual_seed(seed)

# create training and validation dataset
dataset_train, dataset_valid = random_split(Dataset('./data/'), [149,30], generator=torch.Generator().manual_seed(42))
dataset_train, dataset_test = random_split(dataset_train, [121,28], generator=torch.Generator().manual_seed(42))
NWORKERS = 24
device = 'cuda' if torch.cuda.is_available() else 'cpu'

TRAIN = DataLoader(dataset=dataset_train,
                   batch_size=3,
                   shuffle=True,
                   drop_last=False,
                   num_workers=NWORKERS)

TEST = DataLoader(dataset=dataset_test,
                   batch_size=3,
                   shuffle=True,
                   drop_last=False,
                   num_workers=NWORKERS)

VALID = DataLoader(dataset=dataset_valid,
                   batch_size=3,
                   shuffle=True,
                   drop_last=False,
                   num_workers=NWORKERS)




if __name__ == "__main__":
    model = NN().to(DEVICE)
    optimizer = optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-4)
    loss = nn.CrossEntropyLoss()

    for epoch in range(2):
        model.train()
        for i,(x,t) in enumerate(TRAIN):
            optimizer.zero_grad()
            x = torch.stack(x).to(device)
            x= x.float()
            t = t.to(DEVICE).long()
            y = model(x)
            J = loss(input=y,target=t)
            J.backward()
            optimizer.step()

            if i%50==0:
                print('EPOCH:{}\tITER:{}\tLOSS:{}'.format(str(epoch).zfill(2),
                                                          str(i).zfill(5),
                                                          J.data.cpu().numpy()))

        # evaluate results for validation test
        model.eval()
        for i,(x,t) in enumerate(VALID):
            x = x.to(DEVICE).float()
            t = t.to(DEVICE).long()
            y = model(x)
            




In [ ]:
# create training and validation dataset
dataset_train, dataset_valid = random_split(Dataset('./data/'), [149,30], generator=torch.Generator().manual_seed(42))
dataset_train, dataset_test = random_split(dataset_train, [121,28], generator=torch.Generator().manual_seed(42))
NWORKERS = 24
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

TRAIN = DataLoader(dataset=dataset_train,
                   batch_size=32,
                   shuffle=True,
                   drop_last=False,
                   num_workers=NWORKERS)

TEST = DataLoader(dataset=dataset_test,
                   batch_size=32,
                   shuffle=True,
                   drop_last=False,
                   num_workers=NWORKERS)

VALID = DataLoader(dataset=dataset_valid,
                   batch_size=32,
                   shuffle=True,
                   drop_last=False,
                   num_workers=NWORKERS)

for i,(x,t) in enumerate(TRAIN):
    print(x)